In [2]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51",
    "X-Guest-Token": "DnNSb7co65JF9qgP1fzaWtGmRAwkjvEV"
}

In [ ]:
# Read data from CSV file into a dataframe
data_frame = pd.read_csv('data/nha-sach-tiki.csv')

# Get unique seller IDs
unique_sellers = data_frame['seller_id'].unique()

# Create a new dataframe to store seller information
seller_data = pd.DataFrame(columns=['seller_id', 'seller_name'])

# Set the user agent header for the requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Loop through each unique seller ID
for seller_id in unique_sellers:
    # Send a GET request to the API with the corresponding seller ID
    response = requests.get(f'https://tiki.vn/api/shopping/v2/widgets/seller?seller_id={seller_id}', headers=headers)
    
    # Extract the seller name from the API response
    seller_name = response.json()['data']['seller']['name']
    
    # Append the seller ID and name to the seller data dataframe
    seller_data = seller_data.append({'seller_id': seller_id, 'seller_name': seller_name}, ignore_index=True)

# Save the seller data dataframe to a CSV file
seller_data.to_csv('data/seller_list.csv', index=False)

In [ ]:
# Create a dataframe to store category information
category_data = pd.DataFrame(columns=['category_id', 'category_name'])

# Send a GET request to retrieve the initial category listings
response = requests.get(
    'https://tiki.vn/api/personalish/v1/blocks/listings',
    headers=HEADERS,
    params={'category': 8322, 'aggregations': 2}
)

# Extract category information from the API response and append to category_data
for i in pd.DataFrame(response.json()['filters'][0]['values'])['query_value']:
    response = requests.get(
        'https://tiki.vn/api/personalish/v1/blocks/listings',
        headers=HEADERS,
        params={'category': i, 'aggregations': 2}
    )
    for item in response.json()['filters'][0]['values']:
        category_data = category_data.append({'category_id': item['query_value'], 'category_name': item['display_value']}, ignore_index=True)

# Remove duplicate category entries
category_data = category_data.drop_duplicates()

# Read cleaned data from CSV file into a dataframe
data_frame = pd.read_csv('data/nha-sach-tiki-cleaned.csv')

# Get unique primary categories
unique_categories = data_frame['primary_category_path'].unique()

# Create a new DataFrame to store converted categories
converted_categories = pd.DataFrame(columns=['primary_category_path', 'primary_category_name'])

for category in unique_categories:
    category_id = int(category.split("/")[-1])
    for index, row in category_data.iterrows():
        if row['category_id'] == category_id:
            converted_categories = converted_categories.append({
                'primary_category_path': category,
                'primary_category_name': row['category_name']
            }, ignore_index=True)

# Save the converted categories to a CSV file
converted_categories.to_csv('data/converted_categories.csv', index=False)


In [ ]:
# Read cleaned data from a CSV file into a dataframe
df = pd.read_csv('data/nha-sach-tiki-cleaned.csv')

# Get unique primary categories
categories = pd.DataFrame(df['primary_category_path'].unique())
categories.columns = ['primary_category_path']
categories['subcategory'] = categories['primary_category_path'].apply(lambda x: x.split("/")[-1])
categories['category'] = categories['primary_category_path'].apply(lambda x: x.split("/")[-2])
categories['category_name'] = 'Other'

for index, row in categories.iterrows():
    # Send a GET request to retrieve the initial category listings
    response = requests.get(
        'https://tiki.vn/api/personalish/v1/blocks/listings',
        headers=HEADERS,
        params={'category': row['category'], 'aggregations': 2}
    )
    
    for sub in response.json()['filters'][0]['values']:
        if int(sub['query_value']) == int(row['subcategory']):
            categories.loc[index, 'category_name'] = sub['display_value']
            break
categories.to_csv('data/category_list.csv', index=False)

In [29]:
# Read data from CSV file into a dataframe
data_frame = pd.read_csv('data/nha-sach-tiki-cleaned.csv')
#Loại bỏ /t /n trong brand_name
data_frame['brand_name'] = data_frame['brand_name'].str.replace('\n', '')
data_frame['brand_name'] = data_frame['brand_name'].str.replace('\t', '')
data_frame.to_csv('data/nha-sach-tiki-cleaned.csv', index=False)